In [25]:
import os, json

import unstructured_client
from unstructured_client.models import operations, shared

client = unstructured_client.UnstructuredClient(
    api_key_auth=os.getenv("UNSTRUCTURED_API_KEY")
)


In [27]:
unstructured_client.partition

AttributeError: module 'unstructured_client' has no attribute 'partition'

In [ ]:
import os, json
import unstructured_client
from unstructured_client.models import operations, shared
from unstructured_client.models.errors import SDKError
from dotenv import load_dotenv
load_dotenv()

def get_elements_from_file(filename):
    with open(filename, "rb") as f:          # pass bytes, not a file handle
        file_bytes = f.read()
            # print first 10 bytes

    req = operations.PartitionRequest(
        partition_parameters=shared.PartitionParameters(
            strategy=shared.Strategy.FAST,
            files=shared.Files(
                content=file_bytes,
                file_name=filename,
                
            ),
            
            languages=["heb", "eng"],
        )
    )

    client = unstructured_client.UnstructuredClient(
        api_key_auth=os.getenv("UNSTRUCTURED_API_KEY")
    )

    try:  
        res = client.general.partition(request=req)
        elements = res.elements                          # list of Element objects
        
        return elements
    except SDKError as e:                                # friendlier error handling
        print(f"{e.status_code}: {e.message}")


In [2]:
os.getenv("UNSTRUCTURED_API_KEY")

'2CuIov3dF0K9vnm1zu39yJa7QdYqNe'

In [3]:
from pathlib import Path
from typing import List

def list_all_files(start: str | Path) -> List[str]:
    """
    Return a list of full (absolute) paths for every file
    inside *start* and all its sub-directories.

    Parameters
    ----------
    start : str | Path
        The root folder to walk. Tilde (~) is expanded; the
        path is resolved so the result is always absolute.

    Raises
    ------
    FileNotFoundError
        If *start* does not exist or is not a directory.

    Examples
    --------

    """
    root = Path(start).expanduser().resolve()

    if not root.is_dir():
        raise FileNotFoundError(f"{root} is not an existing directory")

    # rglob('*') walks recursively; p.is_file() filters out folders
    return [str(p) for p in root.rglob('*') if p.is_file()]

pathes=list_all_files("docs")
doc=[]
for path in pathes:
    filename=os.path.basename(path)
    if filename.endswith(".doc"):
        print(path)
        doc.append(path)

/Users/matansharon/python/AI/unsructered/docs/Products/Bonded Manifolds/obsolete/Risk Management Plan - Bonded Manifolds.doc
/Users/matansharon/python/AI/unsructered/docs/Products/Bonded Manifolds/obsolete/RMP - Bonded Manifolds.doc
/Users/matansharon/python/AI/unsructered/docs/Products/Disposable PressureTransducer/obsolete/Risk Management Plan - Disposable Pressure Transducer.doc
/Users/matansharon/python/AI/unsructered/docs/Products/Disposable PressureTransducer/obsolete/RMP- Disposable Pressure Transducer modifications.doc
/Users/matansharon/python/AI/unsructered/docs/Products/Closed Stopcock & MRVLS/Obsolete/RMP - Closed Stopcock & MRVLS.doc
/Users/matansharon/python/AI/unsructered/docs/Products/Closed Stopcock & MRVLS/Obsolete/Risk Management Plan - CSTP & MRVLS לא לשימוש.doc


In [18]:
elements=[]
for i in range(1,6):
    el=get_elements_from_file(doc[i])
    elements.append(el)


INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"


In [19]:
for el in elements:
    for e in el:
        print(e)

{'type': 'Header', 'element_id': 'df32bb9499063731ae7c2a472c41b75d', 'text': '3-01-21  SOP 4  Appendix Appendix status - released Revision  5 Valid from \u200f25/12/2016 Product/s name Bonded Manifolds Document status Under Edit Author - Title Revision B Risk Management Plan Release Date - Document# Doc-075384 Page 8 of 8', 'metadata': {'category_depth': 0, 'filename': 'RMP - Bonded Manifolds.doc', 'header_footer_type': 'primary', 'languages': ['eng'], 'filetype': 'application/msword'}}
{'type': 'UncategorizedText', 'element_id': '11e96d58f19dcb0f91ee6f57e9131670', 'text': 'Company: Elcam Medical', 'metadata': {'category_depth': 0, 'emphasized_text_contents': ['Company: Elcam Medical'], 'emphasized_text_tags': ['b'], 'filename': 'RMP - Bonded Manifolds.doc', 'languages': ['eng'], 'filetype': 'application/msword', 'parent_id': 'df32bb9499063731ae7c2a472c41b75d'}}
{'type': 'ListItem', 'element_id': '5761d445000d8a3fbe516913c39b71e0', 'text': 'Product Description:', 'metadata': {'category

In [6]:
import pandas as pd
cols=['type','element_id','text','metadata']
df=pd.DataFrame(columns=cols)
df

,type,element_id,text,metadata


In [12]:
rows_list = []
for element in elements:
    for row in element:
        rows_list.append({
            'type': row['type'],
            'element_id': row['element_id'],
            'text': row['text'],
            'metadata': row['metadata']
        })
df = pd.DataFrame(rows_list, columns=cols)
df

,type,element_id,text,metadata
0,Header,df32bb9499063731ae7c2a472c41b75d,3-01-21 SOP 4 Appendix Appendix status - rel...,"{'category_depth': 0, 'filename': 'RMP - Bonde..."
1,UncategorizedText,11e96d58f19dcb0f91ee6f57e9131670,Company: Elcam Medical,"{'category_depth': 0, 'emphasized_text_content..."
2,ListItem,5761d445000d8a3fbe516913c39b71e0,Product Description:,"{'category_depth': 0, 'emphasized_text_content..."
3,NarrativeText,297a929bb065d15e8a587281927dcce7,The Bonded Manifolds family of products contai...,"{'category_depth': 0, 'filename': 'RMP - Bonde..."
4,NarrativeText,daba200c1a05502b866d26a0844cf190,This Risk management plan covers all risk mana...,"{'category_depth': 0, 'filename': 'RMP - Bonde..."
...,...,...,...,...
444,ListItem,d579f17be9129567cf91899c916f8324,Document Revision Control:,"{'category_depth': 0, 'filename': 'Risk Manage..."
445,Table,7b2bd14544af079deba98293f5e60b11,Revision History Tracking: Date: Revision 0- f...,{'emphasized_text_contents': ['Revision Histor...
446,ListItem,236dcc2021d24a13b2414059db54956f,Review and approval:,"{'category_depth': 0, 'filename': 'Risk Manage..."
447,Table,9a07ed8cb2791352e92addf64419d209,Written by: Name Date Signature QA Eng. - - -,"{'emphasized_text_contents': ['Written by:', '..."


In [16]:
#iterate over the rows of the DataFrame and check if there is any duplicate text
for index, row in df.iterrows():
    if df['text'].duplicated().any():
        print(f"Duplicate text found in row {index}: {row['text']}")
    else:
        print(f"No duplicate text found in row {index}: {row['text']}")


Duplicate text found in row 0: 3-01-21  SOP 4  Appendix Appendix status - released Revision  5 Valid from ‏25/12/2016 Product/s name Bonded Manifolds Document status Under Edit Author - Title Revision B Risk Management Plan Release Date - Document# Doc-075384 Page 8 of 8
Duplicate text found in row 1: Company: Elcam Medical
Duplicate text found in row 2: Product Description:
Duplicate text found in row 3: The Bonded Manifolds family of products contains products that are alternative for Manifold-Monoblocks. They are 2,3 and 4 gang UV bonded manifold, LB and closed stopcocks.  
Duplicate text found in row 4: This Risk management plan covers all risk management activities required for bonded manifolds development projects as well as their design change projects.
Duplicate text found in row 5: Their lifecycle stages of such projects are divided to:
Duplicate text found in row 6: project initiation
Duplicate text found in row 7: Design Planning
Duplicate text found in row 8: Design Input
D

In [24]:
help(unstructured_client.models.operations)

Help on package unstructured_client.models.operations in unstructured_client.models:

NAME
    unstructured_client.models.operations - Code generated by Speakeasy (https://speakeasy.com). DO NOT EDIT.

PACKAGE CONTENTS
    cancel_job
    create_destination
    create_source
    create_workflow
    delete_destination
    delete_source
    delete_workflow
    get_destination
    get_job
    get_source
    get_workflow
    list_destinations
    list_jobs
    list_sources
    list_workflows
    partition
    run_workflow
    update_destination
    update_source
    update_workflow

CLASSES
    builtins.dict(builtins.object)
        unstructured_client.models.operations.cancel_job.CancelJobRequestTypedDict
        unstructured_client.models.operations.cancel_job.CancelJobResponseTypedDict
        unstructured_client.models.operations.create_destination.CreateDestinationRequestTypedDict
        unstructured_client.models.operations.create_destination.CreateDestinationResponseTypedDict
      